In [7]:
import requests
import xml.etree.ElementTree as ET
import pandas as pd
pd.set_option('mode.chained_assignment', 'warn')
def harvest():
    base_url = "http://export.arxiv.org/api/query?"
    url = (base_url +
           "search_query=all:Massachusetts Institute of Technology&" +
           "start=0&" +
           "max_results=3000")
    ns = {
        'atom': 'http://www.w3.org/2005/Atom',
        'arx': 'http://arxiv.org/schemas/atom'
    }
    article_dict = {
        "id": [],
        "title": [],
        "summary": [],
        "published": [],
        "authors": [],
        "primary category": []
    }
    response = requests.get(url)
    root = ET.fromstring(response.text)
    for entry in root.findall("atom:entry", ns):
        article_dict["id"].append(entry.find("atom:id", ns).text)
        article_dict["title"].append(entry.find("atom:title", ns).text)
        article_dict["summary"].append(entry.find("atom:summary", ns).text)
        article_dict["published"].append(entry.find("atom:published", ns).text)
        author_list = []
        for author in entry.findall("atom:author", ns):
            name = author.find("atom:name", ns)
            aff = author.find("arx:affiliation", ns)
            author_list.append(name if name is not None and name.text is not None else 'null'
                                                                                       + ', '
                                                                                       + aff if aff is not None and aff.text is not None else 'null')
        article_dict["authors"].append(author_list)
        article_dict["primary category"].append(entry.find("arx:primary_category", ns).get('term'))
    article_data = pd.DataFrame(article_dict)
    article_data.to_csv('arxiv_mit.csv', index=False) # switched to MIT
harvest()